In [28]:
import json
import pickle

import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [29]:
# Load intents JSON file:
with open("intents.json") as json_file:
    data = json.load(json_file)
    

In [30]:
# A place to store our training data:
training_sentences = []
training_labels = []
labels = []
responses = []


# Loop through each intent:
for intent in data['intents']:
    # Loop through each pattern:
    for pattern in intent['patterns']:
        # Store the pattern:
        training_sentences.append(pattern)
        # Store the associated tag:
        training_labels.append(intent['tag'])

    # Store possible responses:
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])


num_classes = len(labels)

In [31]:
# Implement label encoding:
label_encoder = LabelEncoder()
label_encoder.fit(training_labels)
training_labels = label_encoder.transform(training_labels)


In [32]:
# Vectorize the training data using Tokenization:
vocabulary_size = 1000
embedding_dimension = 16
max_length = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocabulary_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences,
                                 maxlen=max_length,
                                 truncating='post')

In [38]:
# Define Neural Network architecture:
model = Sequential()
model.add(Embedding(vocabulary_size,
                    embedding_dimension,
                    input_length=max_length))
model.add(GlobalAveragePooling1D())
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [39]:
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d_3 ( (None, 16)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_10 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_11 (Dense)             (None, 12)                396       
Total params: 17,996
Trainable params: 17,996
Non-trainable params: 0
_________________________________________________________________


In [40]:
epochs = 500
history = model.fit(padded_sequences,
                    np.array(training_labels),
                    epochs=epochs,
                    workers=4)

Epoch 1/500
3/3 [==============================] - 0s 998us/step - loss: 2.4843 - accuracy: 0.0787
Epoch 2/500
3/3 [==============================] - 0s 662us/step - loss: 2.4815 - accuracy: 0.1461
Epoch 3/500
3/3 [==============================] - 0s 997us/step - loss: 2.4795 - accuracy: 0.1461
Epoch 4/500
3/3 [==============================] - 0s 665us/step - loss: 2.4773 - accuracy: 0.1461
Epoch 5/500
3/3 [==============================] - 0s 662us/step - loss: 2.4756 - accuracy: 0.1461
Epoch 6/500
3/3 [==============================] - 0s 997us/step - loss: 2.4734 - accuracy: 0.1461
Epoch 7/500
3/3 [==============================] - 0s 997us/step - loss: 2.4713 - accuracy: 0.1461
Epoch 8/500
3/3 [==============================] - 0s 997us/step - loss: 2.4692 - accuracy: 0.1461
Epoch 9/500
3/3 [==============================] - 0s 1ms/step - loss: 2.4668 - accuracy: 0.1461
Epoch 10/500
3/3 [==============================] - 0s 1ms/step - loss: 2.4648 - accuracy: 0.1461
Epoch 11/500


In [41]:
# Save the neural network model:
model.save("chat_model")

# Save the fitted tokenizer:
with open("tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle,
                protocol=pickle.HIGHEST_PROTOCOL)
    

# Save the fitted label encoder:
with open("label_encoder.pickle", "wb") as enc_file:
    pickle.dump(label_encoder, enc_file,
                protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets
